# Fine-Tune with PyABSA

## Install required libraries

In [ ]:
# %pip install torch==2.2.2 torchvision torchaudio --upgrade
# %pip install transformers==4.39.3 --upgrade
# %pip install peft==0.10.0 --upgrade
# %pip install pyabsa --upgrade


## Manage libraries

In [ ]:
# %pip uninstall torch torchvision torchaudio

In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

## Check GPU availability

In [ ]:
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")


CUDA Available: False


## PyABSA imports

In [ ]:
from pyabsa import ABSADatasetList, DatasetItem, AspectPolarityClassification as APC, ModelSaveOption, DeviceTypeOption, ATEPCCheckpointManager

## Configure ATE (Aspect Term Extractor)

In [ ]:
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint="english")

inference_source = [
    'The food is good but the service is bad.',
]

atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source)
print(atepc_result)

[2025-04-27 15:55:37] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-27 15:55:37] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-27 15:55:37] (2.4.1.post1) Downloading checkpoint:english 
[2025-04-27 15:55:37] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets


Find zipped checkpoint: ./checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip, unzipping
Done.
[2025-04-27 15:58:40] (2.4.1.post1) If the auto-downloading failed, please download it via browser: https://huggingface.co/spaces/yangheng/PyABSA/resolve/main/checkpoints/English/ATEPC/fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43.zip 
[2025-04-27 15:58:40] (2.4.1.post1) Load aspect extractor from checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43
[2025-04-27 15:58:40] (2.4.1.post1) config: checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.config
[2025-04-27 15:58:40] (2.4.1.post1) state_dict: checkpoints\ATEPC_ENGLISH_CHECKPOINT\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\fast_lcf_atepc.state_dict
[2025-04-27 15:58:40] (2.4.1.post1) model: None
[2025-04-27 15:58:40] (2.4.1.post1) tokenize

c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:593: UserWarning: Cre

[2025-04-27 15:58:48] (2.4.1.post1) The results of aspect term extraction have been saved in d:\FCAIH - Materials\Natural Language Understanding\Project\jakarta_research_sameval_absa\notebooks\Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2025-04-27 15:58:48] (2.4.1.post1) Example 0: The <food:Positive Confidence:0.9952> is good but the <service:Negative Confidence:0.9973> is bad .
[{'sentence': 'The food is good but the service is bad .', 'IOB': ['O', 'B-ASP', 'O', 'O', 'O', 'O', 'B-ASP', 'O', 'O', 'O'], 'tokens': ['The', 'food', 'is', 'good', 'but', 'the', 'service', 'is', 'bad', '.'], 'aspect': ['food', 'service'], 'position': [[1], [6]], 'sentiment': ['Positive', 'Negative'], 'probs': [[0.004458647221326828, 0.0003178288752678782, 0.9952235817909241], [0.997255265712738, 0.0016527267871424556, 0.0010920086642727256]], 'confidence': [0.9952, 0.9973]}]


## Wrapping ATE output with [ASP] tokens

In [21]:
def insert_asp_tokens(sentence, aspect):
    """Insert [ASP] tokens around the aspect term in the sentence."""
    return sentence.replace(aspect, f"[ASP] {aspect} [ASP]", 1)

apc_inputs = []
for result in atepc_result:
    sentence = result['sentence']
    for aspect in result['aspect']:
        formatted_sentence = insert_asp_tokens(sentence, aspect)
        apc_inputs.append(formatted_sentence)

print(apc_inputs)

['The [ASP] food [ASP] is good but the service is bad .', 'The food is good but the [ASP] service [ASP] is bad .']


## Dataset configuration

In [ ]:
config = APC.APCConfigManager.get_apc_config_english()

dataset = ABSADatasetList.Laptop14

# my_dataset = DatasetItem("my_dataset", ["my_dataset1", "my_dataset2"])

## APC (Aspect Polarity Classification) configuration

In [ ]:
config = APC.APCConfigManager.get_apc_config_english()
config.num_epoch = 1
config.model = APC.APCModelList.FAST_LSA_T_V2

# dataset='D:/FCAIH - Materials/Natural Language Understanding/Project/jakarta_research_sameval_absa/datasets/apc_datasets/101.restaurant'

trainer = APC.APCTrainer(
    config=config,
    dataset=dataset,
    from_checkpoint="english",
    # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,
    path_to_save=None,  # set a path to save checkpoints, if it is None, save checkpoints at 'checkpoints' folder
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,
    load_aug=False,
)

[2025-04-27 14:42:05] (2.4.1.post1) Set Model Device: cpu
[2025-04-27 14:42:05] (2.4.1.post1) Device Name: Unknown


c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


2025-04-27 14:42:05,344 INFO: PyABSA version: 2.4.1.post1
2025-04-27 14:42:05,346 INFO: Transformers version: 4.39.3
2025-04-27 14:42:05,347 INFO: Torch version: 2.2.2+cpu+cudaNone
2025-04-27 14:42:05,348 INFO: Device: Unknown
2025-04-27 14:42:05,359 INFO: Searching dataset 113.Laptop14 in local disk
2025-04-27 14:42:05,498 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.
2025-04-27 14:42:05,499 INFO: Please use a new folder to perform new text augment if the former augment in integrated_datasets\apc_datasets\110.SemEval\113.laptop14 errored unexpectedly
[2025-04-27 14:42:05] (2.4.1.post1) Loading dataset cache: fast_lsa_t_v2.Laptop14.dataset.20b920d5f04090745bb62188f5b07715de5535acb34814f724371a92053a0b34.cache


c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


2025-04-27 14:42:14,888 INFO: Model Architecture:
 APCEnsembler(
  (models): ModuleList(
    (0): FAST_LSA_T_V2(
      (bert4global): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128100, 768, padding_idx=0)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): StableDropout()
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-11): 12 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): Linear(in_features=768, out_features=768, bias=True)
                  (key_proj): Linear(in_features=768, out_features=768, bias=True)
                  (value_proj): Linear(in_features=768, out_features=768, bias=True)
                  (pos_dropout): StableDropout()
                  (dropout): StableDropout()
                )
                (output): DebertaV2SelfOutpu

...

Epoch:0 | Loss:0:   0%|          | 0/146 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


[2025-04-27 14:42:19] (2.4.1.post1) reset eta1 to: 0.767673671245575
[2025-04-27 14:42:19] (2.4.1.post1) reset eta2 to: 0.7462112307548523


Epoch:  0 | Smooth Loss: 0.1318: 100%|██████████| 146/146 [46:44<00:00, 19.21s/it, Dev Acc:80.88(max:84.01) Dev F1:76.13(max:80.97)] 


2025-04-27 15:29:01,649 INFO: 
--------------------------------------------------------------------------- Raw Metric Records ---------------------------------------------------------------------------
╒════════════════════════════╤═══════════════════════════════════════════════════════════╤═════════════════════╤═══════════╤══════════╤═══════╤═══════╤═════════╤═════════╕
│ Metric                     │ Trial                                                     │ Values              │  Average  │  Median  │  Std  │  IQR  │   Min   │   Max   │
╞════════════════════════════╪═══════════════════════════════════════════════════════════╪═════════════════════╪═══════════╪══════════╪═══════╪═══════╪═════════╪═════════╡
│ Max-Test-Acc w/o Valid Set │ fast_lsa_t_v2-Laptop14-yangheng/deberta-v3-base-absa-v1.1 │ [84.01253918495298] │  84.0125  │ 84.0125  │   0   │   0   │ 84.0125 │ 84.0125 │
├────────────────────────────┼───────────────────────────────────────────────────────────┼────────────────────

c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyabsa\framework\trainer_class\trainer_template.py:250: ResourceWarning: unclosed file <_io.TextIOWrapper name='d:\\FCAIH - Materials\\Natural Language Understanding\\Project\\jakarta_research_sameval_absa\\notebooks\\logs\\fast_lsa_t_v2_20250427 144042\\trainer.log' mode='a' encoding='utf8'>
  self.config.logger.removeHandler(self.config.logger.handlers[0])


## Loading trained model

In [ ]:
from pyabsa.tasks.AspectPolarityClassification import SentimentClassifier

sentiment_classifier = trainer.load_trained_model()
assert isinstance(sentiment_classifier, SentimentClassifier)

from pyabsa import available_checkpoints

ckpts = available_checkpoints()

sentiment_classifier = APC.SentimentClassifier(
    checkpoint="english"
)

[2025-04-27 15:29:14] (2.4.1.post1) Load sentiment classifier from checkpoints/fast_lsa_t_v2_Laptop14_acc_84.01_f1_80.97/
[2025-04-27 15:29:14] (2.4.1.post1) config: checkpoints/fast_lsa_t_v2_Laptop14_acc_84.01_f1_80.97/fast_lsa_t_v2.config
[2025-04-27 15:29:14] (2.4.1.post1) state_dict: checkpoints/fast_lsa_t_v2_Laptop14_acc_84.01_f1_80.97/fast_lsa_t_v2.state_dict
[2025-04-27 15:29:14] (2.4.1.post1) model: None
[2025-04-27 15:29:14] (2.4.1.post1) tokenizer: checkpoints/fast_lsa_t_v2_Laptop14_acc_84.01_f1_80.97/fast_lsa_t_v2.tokenizer
[2025-04-27 15:29:14] (2.4.1.post1) Set Model Device: cpu
[2025-04-27 15:29:14] (2.4.1.post1) Device Name: Unknown


c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\ALY\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[2025-04-27 15:29:24] (2.4.1.post1) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2025-04-27 15:29:26] (2.4.1.post1) ********** Available APC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-27 15:29:26] (2.4.1.post1) ********** Available APC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-27 15:29:26] (2.4.1.post1) Downloading checkpoint:english 
[2025-04-27 15:29:26] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-04-27 15:29:26] (2.4.1.post1) Checkpoint already downloaded, skip
[2025-04-27 15:29:26] (2.4.1.post1) Load sentiment classifier from checkpoints\APC_ENGLISH_CHECKPOINT\fast_lsa_t_v2_English_acc_82.21_f1_81.81
[2025-04-27 15:29:26] (2.4.1.post1) config: checkpoints\APC_ENGLISH_CHECKPOINT\fast_lsa_t_v2_English_acc_82.21_f1_81.81\fast_lsa_t_v2.config
[2025-04-27 15:29:26] (2.4.1.post1) state_dict: checkpoints\A

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

## Predictions

In [ ]:
apc_output = sentiment_classifier.predict(
    text=apc_inputs,
    print_result=True,
    ignore_error=True,
    eval_batch_size=32,
)

[2025-04-27 16:12:56] (2.4.1.post1) [ASP] tag is detected, please use [B-ASP] and [E-ASP] to annotate aspect terms.
[2025-04-27 16:12:56] (2.4.1.post1) [ASP] tag is detected, please use [B-ASP] and [E-ASP] to annotate aspect terms.
[2025-04-27 16:12:57] (2.4.1.post1) Example 0: The < food :Positive(confidence:0.998, ref:-100)> is good but the< service :Negative(confidence:0.988, ref:-100)>is bad .


## Format the output

In [26]:
def format_atepc_output(results):
    for item in results:
        print(f"\nSentence: {item['text'].strip()}\n")
        for aspect, sentiment, confidence in zip(item['aspect'], item['sentiment'], item['confidence']):
            aspect_clean = aspect.strip()
            confidence_percent = round(confidence * 100, 2)
            print(f"- Aspect: \"{aspect_clean}\" | Sentiment: {sentiment} | Confidence: {confidence_percent}%")

format_atepc_output(apc_output)



Sentence: The  food  is good but the service is bad .

- Aspect: "food" | Sentiment: Positive | Confidence: 99.81%
- Aspect: "service" | Sentiment: Negative | Confidence: 98.82%


In [27]:
def format_atepc_output_custom(results):
    for item in results:
        aspects_info = []
        for aspect, sentiment in zip(item['aspect'], item['sentiment']):
            aspect_clean = aspect.strip()
            sentiment_clean = sentiment.strip()
            aspects_info.append(f'{aspect_clean}: "{sentiment_clean.lower()}"')
        
        aspects_str = ', '.join(aspects_info)
        print(f"Input: {item['text'].strip()}\n")
        print(f"Output: ({aspects_str})\n")

format_atepc_output_custom(apc_output)


Input: The  food  is good but the service is bad .

Output: (food: "positive", service: "negative")



# Applying fine-tuing with LoRA

In [38]:
import math
import torch.nn as nn
from pyabsa.tasks.AspectPolarityClassification import SentimentClassifier

## Loading pre-trained model (Again)

In [59]:
ckpts = available_checkpoints()

sentiment_classifier = APC.SentimentClassifier(
    checkpoint="english"
)

[2025-04-27 18:33:24] (2.4.1.post1) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2025-04-27 18:33:24] (2.4.1.post1) ********** Available APC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-27 18:33:24] (2.4.1.post1) ********** Available APC model checkpoints for Version:2.4.1.post1 (this version) **********
[2025-04-27 18:33:24] (2.4.1.post1) Downloading checkpoint:english 
[2025-04-27 18:33:24] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2025-04-27 18:33:24] (2.4.1.post1) Checkpoint already downloaded, skip
FindFile Warning --> multiple targets ['checkpoints\\APC_ENGLISH_CHECKPOINT\\fast_lsa_t_v2_English_acc_82.21_f1_81.81\\fast_lsa_t_v2.config', 'checkpoints\\ATEPC_ENGLISH_CHECKPOINT\\fast_lcf_atepc_English_cdw_apcacc_82.36_apcf1_81.89_atef1_75.43\\fast_lcf_atepc.config'] found, only return the shortest path: <checkpoints\APC_ENGLISH_CHECKPOINT\

## Define LoRA layer class

In [60]:
class LoRALayer(nn.Module):
    def __init__(self, original_layer, r=8):
        super(LoRALayer, self).__init__()
        self.original_layer = original_layer
        self.r = r
        self.lora_A = nn.Linear(original_layer.in_features, r, bias=False)
        self.lora_B = nn.Linear(r, original_layer.out_features, bias=False)
        # Initialize LoRA weights
        nn.init.kaiming_uniform_(self.lora_A.weight, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B.weight)

    def forward(self, x):
        return self.original_layer(x) + self.lora_B(self.lora_A(x))

## Finding target layer in model architecture

In [61]:
lora_model = sentiment_classifier
print(lora_model.model)

APCEnsembler(
  (models): ModuleList(
    (0): FAST_LSA_T_V2(
      (bert4global): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128100, 768, padding_idx=0)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): StableDropout()
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-11): 12 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): Linear(in_features=768, out_features=768, bias=True)
                  (key_proj): Linear(in_features=768, out_features=768, bias=True)
                  (value_proj): Linear(in_features=768, out_features=768, bias=True)
                  (pos_dropout): StableDropout()
                  (dropout): StableDropout()
                )
                (output): DebertaV2SelfOutput(
                  (dense): Linear(in_features=76

## Access the attention layers for LoRA application

In [62]:
for layer in lora_model.model.models[0].bert4global.encoder.layer:
    # Apply LoRA to each attention projection (query_proj, key_proj, value_proj)
    layer.attention.self.query_proj = LoRALayer(layer.attention.self.query_proj, r=8)
    layer.attention.self.key_proj = LoRALayer(layer.attention.self.key_proj, r=8)
    layer.attention.self.value_proj = LoRALayer(layer.attention.self.value_proj, r=8)


## Adding LoRA layer to target layer

In [65]:
lora_model.model.dense = LoRALayer(lora_model.model.dense, r=8)

## Check new model layers

In [66]:
print(lora_model.model)

APCEnsembler(
  (models): ModuleList(
    (0): FAST_LSA_T_V2(
      (bert4global): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128100, 768, padding_idx=0)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): StableDropout()
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-11): 12 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): LoRALayer(
                    (original_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_A): Linear(in_features=768, out_features=8, bias=False)
                    (lora_B): Linear(in_features=8, out_features=768, bias=False)
                  )
                  (key_proj): LoRALayer(
                    (original_layer): Linear(in_features=768, out_features=768, bias=True)
          

# Training and visualization

In [75]:
import autocuda
import random

from metric_visualizer import MetricVisualizer

from pyabsa import AspectPolarityClassification as APC

import warnings

device = autocuda.auto_cuda()
warnings.filterwarnings('ignore')

seeds = [random.randint(0, 10000) for _ in range(3)]

max_seq_lens = [60, 70, 80, 90, 100]

apc_config_english = APC.APCConfigManager.get_apc_config_english()
apc_config_english.model = APC.APCModelList.FAST_LCF_BERT
apc_config_english.lcf = 'cdw'
apc_config_english.max_seq_len = 80
apc_config_english.cache_dataset = False
apc_config_english.patience = 10
apc_config_english.seed = seeds

config = APC.APCConfigManager.get_apc_config_english()
config.num_epoch = 1
config.model = APC.APCModelList.FAST_LSA_T_V2

MV = MetricVisualizer(name='lora_model')
apc_config_english.MV = MV

for eta in max_seq_lens:
    apc_config_english.eta = eta
    dataset = APC.APCDatasetList.Laptop14
    APC.APCTrainer(config=config,
                   dataset=dataset,  # train set and test set will be automatically detected
                   checkpoint_save_mode=0,  # =None to avoid save model
                   auto_device=device  # automatic choose CUDA or CPU
                   )
    apc_config_english.MV.next_trial()

save_prefix = '{}_{}'.format(apc_config_english.model_name, apc_config_english.dataset_name)

MV.summary(save_path=save_prefix, no_print=True)  # save fig_preview into .tex and .pdf format
MV.traj_plot_by_trial(save_path=save_prefix, xlabel='', xrotation=30,
                      minorticks_on=True)  # save fig_preview into .tex and .pdf format
MV.violin_plot_by_trial(save_path=save_prefix, xticks=max_seq_lens,
                        xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.box_plot_by_trial(save_path=save_prefix, xticks=max_seq_lens,
                     xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.avg_bar_plot_by_trial(save_path=save_prefix, xticks=max_seq_lens,
                         xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.sum_bar_plot_by_trial(save_path=save_prefix, xticks=max_seq_lens,
                         xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.scott_knott_plot(save_path=save_prefix, minorticks_on=False, xticks=max_seq_lens,
                    xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format

# print(MV.rank_test_by_trail('trial0'))  # save fig_preview into .tex and .pdf format
# print(MV.rank_test_by_metric('metric1'))  # save fig_preview into .tex and .pdf format

[2025-04-27 18:39:28] (2.4.1.post1) Set Model Device: cpu
[2025-04-27 18:39:28] (2.4.1.post1) Device Name: Unknown
2025-04-27 18:39:29,144 INFO: PyABSA version: 2.4.1.post1
2025-04-27 18:39:29,146 INFO: Transformers version: 4.39.3
2025-04-27 18:39:29,147 INFO: Torch version: 2.2.2+cpu+cudaNone
2025-04-27 18:39:29,148 INFO: Device: Unknown
2025-04-27 18:39:29,220 INFO: Searching dataset 113.Laptop14 in local disk
2025-04-27 18:39:29,337 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.
2025-04-27 18:39:29,338 INFO: Please use a new folder to perform new text augment if the former augment in integrated_datasets\apc_datasets\110.SemEval\113.laptop14 errored unexpectedly
[2025-04-27 18:39:29] (2.4.1.post1) Loading dataset cache: fast_lsa_t_v2.Laptop14.dataset.cf68cf1eebd128ab3fee3d23cf6ac43daddf54975a199e4e32e60f650e112f28.cache
2025-04-27 18:39:38,961 INFO: Model Architecture:
 APCEnsembler(
  (models): ModuleList(
    (0): F

Epoch:  0 | Smooth Loss: 0.2367: 100%|██████████| 146/146 [47:35<00:00, 19.56s/it, Dev Acc:81.97(max:83.86) Dev F1:78.23(max:80.47)] 


2025-04-27 19:27:14,569 INFO: 
--------------------------------------------------------------------------- Raw Metric Records ---------------------------------------------------------------------------
╒════════════════════════════╤═══════════════════════════════════════════════════════════╤═════════════════════╤═══════════╤══════════╤═══════╤═══════╤═════════╤═════════╕
│ Metric                     │ Trial                                                     │ Values              │  Average  │  Median  │  Std  │  IQR  │   Min   │   Max   │
╞════════════════════════════╪═══════════════════════════════════════════════════════════╪═════════════════════╪═══════════╪══════════╪═══════╪═══════╪═════════╪═════════╡
│ Max-Test-Acc w/o Valid Set │ fast_lsa_t_v2-Laptop14-yangheng/deberta-v3-base-absa-v1.1 │ [83.85579937304075] │  83.8558  │ 83.8558  │   0   │   0   │ 83.8558 │ 83.8558 │
├────────────────────────────┼───────────────────────────────────────────────────────────┼────────────────────

preparing dataloader: 100%|██████████| 2328/2328 [00:01<00:00, 1716.22it/s]


2025-04-27 19:27:25,332 INFO: Dataset Label Details: {'Negative': 870, 'Positive': 994, 'Neutral': 464, 'Sum': 2328}
2025-04-27 19:27:26,272 INFO: train data examples:
 [{'ex_id': tensor(0), 'text_raw': 'I charge it at night and skip taking the cord with me because of the good battery life .', 'text_spc': '[CLS] I charge it at night and skip taking the cord with me because of the good battery life . [SEP] cord [SEP]', 'aspect': 'cord', 'aspect_position': tensor(0, dtype=torch.int32), 'lca_ids': tensor([0.6667, 0.7143, 0.7619, 0.8095, 0.8571, 0.9048, 0.9524, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9524, 0.9048, 0.8571, 0.8095,
        0.7619, 0.7143, 0.6667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.00

preparing dataloader: 100%|██████████| 638/638 [00:00<00:00, 2100.30it/s]


2025-04-27 19:27:27,072 INFO: Dataset Label Details: {'Negative': 128, 'Positive': 341, 'Neutral': 169, 'Sum': 638}
2025-04-27 19:27:27,336 INFO: test data examples:
 [{'ex_id': tensor(0), 'text_raw': ' Boot time is super fast , around anywhere from 35 seconds to 1 minute .', 'text_spc': '[CLS]  Boot time is super fast , around anywhere from 35 seconds to 1 minute . [SEP] Boot time [SEP]', 'aspect': 'Boot time', 'aspect_position': tensor(0, dtype=torch.int32), 'lca_ids': tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9412, 0.8824, 0.8235,
        0.7647, 0.7059, 0.6471, 0.5882, 0.5294, 0.4706, 0.4118, 0.3529, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.00

Epoch:  0 | Smooth Loss: 0.7124:   8%|▊         | 11/146 [04:00<49:12, 21.87s/it, Dev Acc:83.86(max:83.86) Dev F1:80.04(max:80.04)]  


KeyboardInterrupt: 

## Testing model's output

In [76]:
lora_output = lora_model.predict(
    text=apc_inputs,
    print_result=True,
    ignore_error=True,
    eval_batch_size=32,
)

[2025-04-27 19:31:43] (2.4.1.post1) [ASP] tag is detected, please use [B-ASP] and [E-ASP] to annotate aspect terms.
[2025-04-27 19:31:43] (2.4.1.post1) [ASP] tag is detected, please use [B-ASP] and [E-ASP] to annotate aspect terms.
[2025-04-27 19:31:45] (2.4.1.post1) Example 0: The < food :Positive(confidence:0.998, ref:-100)> is good but the< service :Negative(confidence:0.988, ref:-100)>is bad .


## Format the output

In [77]:
def format_atepc_output(results):
    for item in results:
        print(f"\nSentence: {item['text'].strip()}\n")
        for aspect, sentiment, confidence in zip(item['aspect'], item['sentiment'], item['confidence']):
            aspect_clean = aspect.strip()
            confidence_percent = round(confidence * 100, 2)
            print(f"- Aspect: \"{aspect_clean}\" | Sentiment: {sentiment} | Confidence: {confidence_percent}%")

format_atepc_output(apc_output)



Sentence: The  food  is good but the service is bad .

- Aspect: "food" | Sentiment: Positive | Confidence: 99.81%
- Aspect: "service" | Sentiment: Negative | Confidence: 98.82%


In [78]:
def format_atepc_output_custom(results):
    for item in results:
        aspects_info = []
        for aspect, sentiment in zip(item['aspect'], item['sentiment']):
            aspect_clean = aspect.strip()
            sentiment_clean = sentiment.strip()
            aspects_info.append(f'{aspect_clean}: "{sentiment_clean.lower()}"')
        
        aspects_str = ', '.join(aspects_info)
        print(f"Input: {item['text'].strip()}\n")
        print(f"Output: ({aspects_str})\n")

format_atepc_output_custom(apc_output)


Input: The  food  is good but the service is bad .

Output: (food: "positive", service: "negative")

